In [ ]:
import time
import scipy.ndimage as nd
import scipy.misc
import cv2
import numpy as np
import os
import glob
import gc
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR

import plotly.graph_objects as go
%matplotlib inline

In [ ]:
# import random as rn
# np.random.seed(4321)
# rn.seed(4321)
# tf.random.set_seed(4321)

In [ ]:
! pip install gdown
! gdown https://drive.google.com/uc?id=1CP4vJekB3ryEupq_E7qf41dswengMYhb

In [ ]:
def SSIMLoss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

def custom_loss(y_true, y_pred):
    alpha = 0.84
    loss_ssim = SSIMLoss(y_true, y_pred)    
    logcosh = tf.keras.losses.logcosh(y_true, y_pred)
    
    return (1-alpha)*loss_ssim + alpha*logcosh

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model('./sha-sat-70_30_downstream.h5', compile=False) # custom_objects ={'loss': custom_loss}

In [ ]:
def get_random_crop(image, crop_height, crop_width):
#     print(image.shape)
    max_x = image.shape[0] - crop_width
    max_y = image.shape[1] - crop_height
#     print(max_x, max_y)
    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)
    crop = image[y: y + crop_height, x: x + crop_width,]
    
#     print(crop.shape)
    return crop

def get_center_crop(image, crop_height, crop_width):
#     print(image.shape)
    max_x = image.shape[0] - crop_width
    max_y = image.shape[1] - crop_height
#     print(max_x, max_y)
    x1, x2 = int(max_x/2), int(image.shape[0] - max_x/2)
    y1, y2 = int(max_y/2), int(image.shape[1] - max_y/2)
    print(x1, x2, y1, y2)
    crop = image[y1: y2, x1: x2, ]
    
#     print(crop.shape)
    return crop

In [ ]:
# Read images and masks
nos = [31, 46, 99, 105, 110, 114, 118, 129, 143, 155]
img_dir = "../input/celebahq-resized-256x256/celeba_hq_256/"
mask_dir = "../input/qd-imd-30-50/qd_imd/test/"

imgs = []
masks = []
for img_no in nos:
    img = cv2.imread(img_dir+str(img_no).zfill(5)+'.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = get_center_crop(img, 224, 224)
    imgs.append(img)

for mask_no in nos:
    mask = cv2.imread(mask_dir+str(mask_no).zfill(5)+'_test.png')
    masks.append(mask)

print(len(imgs), len(masks))

In [ ]:
# fig, axs = plt.subplots(nrows=len(imgs), ncols=1, figsize=(6, 2*rows))
# for i in range(len(imgs)):
#     print(imgs[i].shape)
#     axs[i].imshow(imgs[i])

# fig, axs = plt.subplots(nrows=len(masks), ncols=1, figsize=(6, 2*rows))
# for i in range(len(masks)):
#     print(masks[i].shape)
#     axs[i].imshow(masks[i])

# plt.show()

In [ ]:
# Compute SSIM and PSNR for all
rows = len(imgs)
fig, axs = plt.subplots(nrows=rows, ncols=3, figsize=(6, 3*rows))
for i in range(len(imgs)):
    masked_img = cv2.bitwise_and(imgs[i], masks[i])
    inpainted_image = model.predict(masked_img.reshape((1,)+masked_img.shape)/255.0)
    inpainted_image = inpainted_image.reshape(inpainted_image.shape[1:])
#         inpainted_image *= 255.0 
    s = SSIM(imgs[i], inpainted_image*255.0, multichannel = True)

    axs[i][0].imshow(imgs[i]); axs[i][0].set_axis_off();
    axs[i][1].imshow(masked_img); axs[i][1].set_axis_off();
    axs[i][2].imshow(inpainted_image); axs[i][2].set_axis_off();
    print(s)
    
#     cv2.imwrite(str(nos[i]).zfill(5)+'.png', inpainted_image*255.0)
    cv2.imwrite(str(nos[i]).zfill(5)+'.png', cv2.cvtColor(inpainted_image, cv2.COLOR_RGB2BGR)*255.0)

plt.show()

In [ ]:
! mkdir original
for img_no in nos:
    img = cv2.imread(img_dir+str(img_no).zfill(5)+'.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = get_center_crop(img, 224, 224)
    
    cv2.imwrite('./original/' + str(img_no).zfill(5)+'.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))